In [1]:
#reload the modules before running the program
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy
from scipy import signal
from scipy.fft import fftshift
from scipy.fft import fft, fftfreq

import matplotlib.pyplot as plt
import matplotlib as mpl

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import os
import math
import cv2

from datasets import RFData, CameraData, SignalDataset, FuseDatasets
import metrics
import utils

In [2]:
np.random.seed(0)
torch.manual_seed(0)
data_path = r"E:\OSA_project\OSA_project\Datasets\osa_new_processed"
# trial_folders = ["v_12_5", "v_13_5", "v_13_10"]
trial_folders = os.listdir(data_path)

# train_idxs = np.linspace(0, 49, 50, dtype=int)
train_folders = trial_folders#[trial_folders[i] for i in train_idxs]
# test_folders = [trial_folders[i] for i in range(len(trial_folders)) if i not in train_idxs]


thermal_file_name = "Thermal_Camera"
num_samps_oversample = 20 # per experiment, number of samples to generate
data_length = 9000
fs = 30
out_len = 1800 # sample length generated
thermal_ext = ".tiff"

dataset_thermal_train = CameraData(data_path, train_folders, thermal_file_name, num_samps_oversample, fs, data_length, out_len, thermal_ext)
# dataset_thermal_test = CameraData(data_path, test_folders, thermal_file_name, num_samps_oversample, fs, data_length, out_len, thermal_ext)

samp_f=5e6
freq_slope=60.012e12
samples_per_chirp=256
num_tx = 3
num_rx = 4
radar_file_name = "FMCW_Radar.npy"
window_size = 5 # number of range bins to use

dataset_radar_train = RFData(data_path, train_folders, data_length, radar_file_name, out_len, window_size, samples_per_chirp, samp_f, freq_slope, num_samps_oversample, num_tx, num_rx, fs)
# dataset_radar_test = RFData(data_path, test_folders, data_length, radar_file_name, out_len, window_size, samples_per_chirp, samp_f, freq_slope, num_samps_oversample, num_tx, num_rx, fs)

In [3]:
np.random.seed(0)
torch.manual_seed(0)
vital_dict_file_name = "gt_dict.pkl"
vital_key_radar = "CHEST"
vital_key_thermal = "AIR_flow"
l_freq_bpm = 5
u_freq_bpm = 30
dataset_OSA_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'OSA', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_CSA_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'CSA', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_MSA_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'MSA', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_hypopnea_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'Hypopnea', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_partial_apnea_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'Partial_Apnea', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_gt_thermal_train = SignalDataset(data_path, train_folders, vital_dict_file_name, vital_key_thermal, data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)
dataset_gt_radar_train = SignalDataset(data_path, train_folders, vital_dict_file_name, vital_key_radar, data_length, out_len, True, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)

# dataset_apnea_test = SignalDataset(data_path, test_folders, vital_dict_file_name, 'Sleep_Apnea', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
#                               normalize=False)
# dataset_gt_thermal_test = SignalDataset(data_path, test_folders, vital_dict_file_name, vital_key_thermal, data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)
# dataset_gt_radar_test = SignalDataset(data_path, test_folders, vital_dict_file_name, vital_key_radar, data_length, out_len, True, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)

KeyError: 'OSA'

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
fused_dataset_train = FuseDatasets([dataset_radar_train, dataset_thermal_train, dataset_gt_radar_train, dataset_gt_thermal_train, dataset_OSA_train, dataset_CSA_train, dataset_hypopnea_train, dataset_partial_apnea_train], ["radar", "thermal", "gt_radar", "gt_ir", "gt_OSA", "gt_CSA", "hypopnea", "partial_apnea"], out_len=out_len)
# fused_dataset_test = FuseDatasets([dataset_radar_test, dataset_thermal_test, dataset_gt_radar_test, dataset_gt_thermal_test, dataset_apnea_test], ["radar", "thermal", "gt_radar", "gt_ir", "gt_apnea"], out_len=out_len)
t_arr = np.linspace(0, out_len/fs, out_len)

In [ ]:
# dmax = int(2*((len(lmax))/17)**(1/2))

In [ ]:
def hl_envelopes_idx(s, dmin_num=1, dmax_num=1, dmax_func=None, dmin_func=None, step_size=1, display=False):
    """
    Input :
    s: 1d-array, data signal from which to extract high and low envelopes
    dmin, dmax: int, optional, size of chunks, use this if the size of the input signal is too big
    split: bool, optional, if True, split the signal in half along its mean, might help to generate the envelope in some cases
    Output :
    lmin,lmax : high/low envelope idx of input signal s
    """
    lmin = (np.diff(np.sign(np.diff(s))) > 0).nonzero()[0] + 1 
    lmax = (np.diff(np.sign(np.diff(s))) < 0).nonzero()[0] + 1

    dmin = dmin_num
    dmax = dmax_num

    if(dmax_func is not None):
        dmax = dmax_func(lmax)

    if(dmin_func is not None):
        print('len(lmin): ', len(lmin), '\n')
        dmin = dmin_func(lmin)
    
    if(display == True):
        print('len(lmax): ', len(lmax), '\n')
        print('len(lmin): ', len(lmin), '\n')
        print('step size: ', step_size, '\n')

    lmin = lmin[[max(i-dmin//2,0)+np.argmin(s[lmin[max(i-dmin//2,0):min(i+dmin//2,len(lmin))]]) for i in range(0,len(lmin),step_size)]]
    lmax = lmax[[max(i-dmax//2,0)+np.argmax(s[lmax[max(i-dmax//2,0):min(i+dmax//2,len(lmax))]]) for i in range(0,len(lmax),step_size)]]
    
    return lmin,lmax